In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:

date=spark.sql("""SELECT 
DATE_SUB(
    DATE_TRUNC('MONTH', ADD_MONTHS(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)), 7) +
CASE 
    WHEN 4 - DAYOFWEEK(DATE_TRUNC('MONTH', ADD_MONTHS(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1))) < 0 
         THEN 4 - DAYOFWEEK(DATE_TRUNC('MONTH', ADD_MONTHS(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1))) + 7
    ELSE 4 - DAYOFWEEK(DATE_TRUNC('MONTH', ADD_MONTHS(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1)))
END AS DATE""").collect()[0]['DATE']


In [0]:
if date == spark.sql("SELECT DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)").collect()[0][0]:
    spark.sql(f"""
    CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_fresh.d_store_group_temp
    -- (
    --     calday DATE --OPTIONS (DESCRIPTION = 'Ngày tính'),
    --     business_unit STRING --OPTIONS (DESCRIPTION = 'Mã chuỗi siêu thị'),
    --     store_id STRING --OPTIONS (DESCRIPTION = 'Mã cửa hàng'),
    --     store_name STRING --OPTIONS (DESCRIPTION = 'Tên cửa hàng'),
    --     concept STRING --OPTIONS (DESCRIPTION = 'Tên loại chuỗi cửa hàng'),
    --     region STRING --OPTIONS (DESCRIPTION = 'Miền'),
    --     base_sale_qty_avg DOUBLE --OPTIONS (DESCRIPTION = 'Số đơn vị bán bình quân ngày tính trong 4 tuần gần nhất'),
    --     sale_kg DOUBLE --OPTIONS (DESCRIPTION = 'Quy đổi số kg rau củ quả bình quân ngày tính trong 4 tuần gần nhất'),
    --     store_group_name STRING --OPTIONS (DESCRIPTION = 'Mô tả phân nhóm cửa hàng theo SALE_KG'),
    --     store_group_id STRING --OPTIONS (DESCRIPTION = 'Phân nhóm cửa hàng theo SALE_KG')
    -- )
   -- CLUSTER BY store_id
   -- OPTIONS ( DESCRIPTION = "Bảng suggest tính phân loại cửa hàng theo sức bán bình quân ngày" )
    AS
    WITH
    store_sku AS (
    SELECT
      s.business_unit,
      s.store_id,
      s.store_name,
      -- sku.product_id,
      CASE
        WHEN business_unit = '1500' THEN 'Supermarket'
        WHEN business_unit = '2000' AND concept IN('WiN','Rural') THEN concept
        ELSE 'Urban'
      END AS concept,
      s.mien AS region,
      SUM(sku.revenue.base_sale_qty)/28 AS base_sale_qty_avg,
      SUM(sku.revenue.base_sale_qty * pm.conversion_base_kg)/28 AS sale_kg
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store s
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily sku ON s.store_id = sku.store_id
      AND sku.calday BETWEEN date_add(date_trunc('week', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), - 7 * 4) AND date_add(date_trunc('week', DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)), - 1)
    JOIN {catalog_name}.udp_wcm_gold_db_winare_fresh.d_product_planning_mapping pm ON sku.product_id = pm.product_id
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product p ON sku.product_id = p.product_id AND p.mch3_id = '10103'
    WHERE 1=1
      AND s.business_unit = '2000'
      AND s.store_status_desc IN ('Đang hoạt động', 'Sắp đóng cửa')
    GROUP BY 1,2,3,4,5
    ),
    main AS (
    SELECT
      ss.business_unit,
      ss.store_id,
      ss.store_name,
      ss.concept,
      ss.region,
      ss.base_sale_qty_avg,
      ss.sale_kg,
      cf.store_group_name,
      cf.store_group_id
    FROM store_sku ss
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_fresh.d_cfg_store_group cf ON ss.concept = cf.concept
      AND ss.sale_kg >= cf.min_qty AND ss.sale_kg < cf.max_qty
    )
    SELECT DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
      m.business_unit,
      m.store_id,
      m.store_name,
      m.concept,
      m.region,
      m.base_sale_qty_avg,
      m.sale_kg,
      IFNULL(cf2.store_group_name, m.store_group_name) AS store_group_name,
      IF(m.store_group_id IS NULL, 'N1', m.store_group_id) AS store_group_id
      FROM main m
      LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_fresh.d_cfg_store_group cf2 ON IF(m.store_group_id IS NULL, 'N1', m.store_group_id) = cf2.store_group_id
      AND m.concept = cf2.concept AND cf2.store_group_id = 'N1'
    ;
""")